# Web Scrape the ESPN Soccer teams
## Author: Pedro Sanhueza
## Website: [www.foxsports.com](https://www.foxsports.com/soccer/2022-fifa-world-cup/teams)

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
from datetime import datetime

In [2]:
url_main = 'https://www.foxsports.com/soccer/2022-fifa-world-cup/teams'
response = requests.get(url_main)
soup = BeautifulSoup(response.text, 'html.parser')

In [3]:
country_name = [x.text for x in soup.find_all('h3')]
countries = [x['href'] for x in soup.find_all('a',{'class':'entity-list-row-container image-logo'})]

In [4]:
country = countries[10]

url = f'https://www.foxsports.com{country}-roster'
soup = BeautifulSoup(requests.get(url).text, 'html.parser')

table = soup.find('div',{'view':'team'})
title = [x.text.strip() for x in table.find_all('tr')[0]]

In [5]:
# running time: 33s -to- 1m 30s-to- 3m 05s

rows=[]
for idx,country in enumerate(countries):

    url = f'https://www.foxsports.com{country}-roster'
    soup = BeautifulSoup(requests.get(url).text, 'html.parser')

    table = soup.find('div',{'view':'team'})
    title = [x.text.strip() for x in table.find_all('tr')[0]]
    
    for group in table.find_all('tbody')[:-1]:
        for player in group:
            row={}
            row['Country'] = country_name[idx].capitalize()
            row['Name'] = player.find('h3').text
            row[title[1]] = player.find('td',{'data-index':'1'}).text.strip()
            row[title[2]] = player.find('td',{'data-index':'2'}).text.strip()
            row[title[3]] = player.find('td',{'data-index':'3'}).text.strip()
            row[title[4]] = player.find('td',{'data-index':'4'}).text.strip()
            rows.append(row)

In [6]:
data = pd.DataFrame(rows)
data


,Country,Name,POS,AGE,HT,WT
0,Argentina,Franco Armani,G,36,"6'2""",194 lbs
1,Argentina,Emiliano Martinez,G,30,"6'4""",196 lbs
2,Argentina,Geronimo Rulli,G,30,"6'2""",185 lbs
3,Argentina,Juan Foyth,D,24,"6'2""",152 lbs
4,Argentina,Lisandro Martinez,D,24,"5'8""",158 lbs
...,...,...,...,...,...,...
826,Wales,Gareth Bale,F,33,"6'0""",180 lbs
827,Wales,Mark Harris,F,23,"5'11""",165 lbs
828,Wales,Daniel James,F,25,"5'7""",167 lbs
829,Wales,Brennan Johnson,F,21,"5'10""",160 lbs


In [7]:
date = datetime.now().strftime("%d-%m-%Y %H%M%S") # get local time as string

filePath = f"../FIFA - Output/{url_main.split('/')[-2]} {date} - Webscrape.csv"

data.to_csv(filePath, index=False) # save to file path

pd.set_option('display.max_rows', 100)

data

,Country,Name,POS,AGE,HT,WT
0,Argentina,Franco Armani,G,36,"6'2""",194 lbs
1,Argentina,Emiliano Martinez,G,30,"6'4""",196 lbs
2,Argentina,Geronimo Rulli,G,30,"6'2""",185 lbs
3,Argentina,Juan Foyth,D,24,"6'2""",152 lbs
4,Argentina,Lisandro Martinez,D,24,"5'8""",158 lbs
...,...,...,...,...,...,...
826,Wales,Gareth Bale,F,33,"6'0""",180 lbs
827,Wales,Mark Harris,F,23,"5'11""",165 lbs
828,Wales,Daniel James,F,25,"5'7""",167 lbs
829,Wales,Brennan Johnson,F,21,"5'10""",160 lbs


In [8]:
# Clean table

# TO DO:
# drop duplicated players within a country (ex: Argentina 'Agustin Rossi')

# build dataframe
data = pd.DataFrame(rows)

# drop rows with missing data
data = data[~data.isin(['-']).any(axis=1)]

# replace possition values
# POS_values = {x.text.strip().capitalize() for x in table.find_all('th',{'data-index':'0'})}
# POS_keys = set(data[title[1]].to_list())
# POS_mapped = dict(zip(POS_keys, POS_values))
POS_mapped = {'G': 'Goalkeeper', 'D': 'Defender', 'M': 'Midfielder', 'F': 'Forward'}
data.replace({title[1]: POS_mapped}, inplace=True)

# age column to number
data[title[2]] = pd.to_numeric(data[title[2]])

# hight column to number
data[title[3]] = data[title[3]].apply(lambda x: (int(x.split('\'')[0])*12 + int(x.split('\'')[1].replace('\"',''))) * 2.54) # inches to centimeters

# weight column to number
data[title[4]] = data[title[4]].apply(lambda x: round(int(x.split(' ')[0]) / 2.205,1)) # lbs to kg

data


,Country,Name,POS,AGE,HT,WT
0,Argentina,Franco Armani,Goalkeeper,36,187.96,88.0
1,Argentina,Emiliano Martinez,Goalkeeper,30,193.04,88.9
2,Argentina,Geronimo Rulli,Goalkeeper,30,187.96,83.9
3,Argentina,Juan Foyth,Defender,24,187.96,68.9
4,Argentina,Lisandro Martinez,Defender,24,172.72,71.7
...,...,...,...,...,...,...
826,Wales,Gareth Bale,Forward,33,182.88,81.6
827,Wales,Mark Harris,Forward,23,180.34,74.8
828,Wales,Daniel James,Forward,25,170.18,75.7
829,Wales,Brennan Johnson,Forward,21,177.80,72.6


In [11]:
data.to_csv(filePath, index=False) # save to file path

In [9]:
# Average per possition
data.groupby('POS').mean().round(1)

<ipython-input-9-96c80662f92a>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data.groupby('POS').mean().round(1)


,AGE,HT,WT
POS,,,
Defender,27.0,181.8,76.4
Forward,26.6,179.7,74.7
Goalkeeper,29.6,188.6,82.6
Midfielder,26.3,178.5,72.2


In [10]:
data.groupby('Country').mean().round(1)

<ipython-input-10-518b7c706663>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data.groupby('Country').mean().round(1)


,AGE,HT,WT
Country,,,
Argentina,27.8,177.5,73.5
Australia,27.4,180.2,75.8
Belgium,27.8,183.3,76.6
Brazil,27.9,179.6,74.1
Cameroon,26.3,180.5,77.9
Canada,27.0,180.6,76.3
Costa rica,28.0,181.1,74.9
Croatia,27.4,183.3,76.9
Denmark,27.2,184.5,78.4


In [ ]:
# which country has the maximun average hight in their golee? 

data1 = data.groupby(['Country','POS']).mean().round(1).reset_index()

data1 = data1[data1.POS == 'Goalkeeper']

# data1[data1.HT == data1.HT.max()]

data1

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
from datetime import datetime

url_main = 'https://www.foxsports.com/soccer/2022-fifa-world-cup/teams'
response = requests.get(url_main)
soup = BeautifulSoup(response.text, 'html.parser')
country_name = [x.text for x in soup.find_all('h3')]
countries = [x['href'] for x in soup.find_all('a',{'class':'entity-list-row-container image-logo'})]


# ---------------------------------------- running time: 1m 20s ---------------------------------------- #

rows=[]

for idx,country in enumerate(countries):

    url = f'https://www.foxsports.com{country}-roster'
    soup = BeautifulSoup(requests.get(url).text, 'html.parser')

    # image = soup.find('source')['srcset']
    table = soup.find('div',{'view':'team'})
    title = [x.text.strip() for x in table.find_all('tr')[0]]
    
    for group in table.find_all('tbody')[:-1]:
        for player in group:
            row={}
            row['Country'] = country_name[idx].capitalize()
            row['Name'] = player.find('h3').text
            row[title[1]] = player.find('td',{'data-index':'1'}).text.strip()
            row[title[2]] = player.find('td',{'data-index':'2'}).text.strip()
            row[title[3]] = player.find('td',{'data-index':'3'}).text.strip()
            row[title[4]] = player.find('td',{'data-index':'4'}).text.strip()
            row['Country_logo'] = soup.find('source')['srcset']
            rows.append(row)

data = pd.DataFrame(rows)

data = data[~data.isin(['-']).any(axis=1)] # drop rows with missing data

POS_mapped = {'G': 'Goalkeeper', 'D': 'Defender', 'M': 'Midfielder', 'F': 'Forward'}

data.replace({title[1]: POS_mapped}, inplace=True)

data[title[2]] = pd.to_numeric(data[title[2]]) # age column to number

data[title[3]] = data[title[3]].apply(lambda x: (int(x.split('\'')[0])*12 + int(x.split('\'')[1].replace('\"',''))) * 2.54) # inches to centimeters

# weight column to number
data[title[4]] = data[title[4]].apply(lambda x: round(int(x.split(' ')[0]) / 2.205,1)) # lbs to kg

# add BMI column
data['BMI'] = data.apply(lambda x: round(x.WT / (x.HT/100)**2,1) , axis=1)

In [5]:
data
# amount of countries
# Per Country
    # Number of players
    # Number of players per position
    # Average age
    # Average HT
    # Average WT

,Country,Name,POS,AGE,HT,WT,Country_logo,BMI
0,Argentina,Franco Armani,Goalkeeper,36,187.96,88.0,https://b.fssta.com/uploads/application/countr...,24.9
1,Argentina,Emiliano Martinez,Goalkeeper,30,193.04,88.9,https://b.fssta.com/uploads/application/countr...,23.9
2,Argentina,Geronimo Rulli,Goalkeeper,30,187.96,83.9,https://b.fssta.com/uploads/application/countr...,23.7
3,Argentina,Juan Foyth,Defender,24,187.96,68.9,https://b.fssta.com/uploads/application/countr...,19.5
4,Argentina,Lisandro Martinez,Defender,24,172.72,71.7,https://b.fssta.com/uploads/application/countr...,24.0
...,...,...,...,...,...,...,...,...
748,Wales,Gareth Bale,Forward,33,182.88,81.6,https://b.fssta.com/uploads/application/countr...,24.4
749,Wales,Mark Harris,Forward,23,180.34,74.8,https://b.fssta.com/uploads/application/countr...,23.0
750,Wales,Daniel James,Forward,25,170.18,75.7,https://b.fssta.com/uploads/application/countr...,26.1
751,Wales,Brennan Johnson,Forward,21,177.80,72.6,https://b.fssta.com/uploads/application/countr...,23.0


In [6]:
# data.to_csv("2022-fifa-world-cup.csv")

In [11]:
(data.POS == 'Goalkeeper').sum()

89

In [77]:
df_c = data[data.Country == "Canada"]
df_c

,Country,Name,POS,AGE,HT,WT,Country_logo,BMI
130,Canada,Milan Borjan,Goalkeeper,35,193.04,82.5,https://b.fssta.com/uploads/application/countr...,22.1
131,Canada,James Pantemis,Goalkeeper,25,182.88,75.7,https://b.fssta.com/uploads/application/countr...,22.6
132,Canada,Dayne St. Clair,Goalkeeper,25,190.50,74.8,https://b.fssta.com/uploads/application/countr...,20.6
133,Canada,Sam Adekugbe,Defender,27,172.72,74.8,https://b.fssta.com/uploads/application/countr...,25.1
134,Canada,Derek Cornelius,Defender,25,185.42,85.7,https://b.fssta.com/uploads/application/countr...,24.9
135,Canada,Alistair Johnston,Defender,24,177.80,76.6,https://b.fssta.com/uploads/application/countr...,24.2
136,Canada,Richie Laryea,Defender,27,172.72,67.6,https://b.fssta.com/uploads/application/countr...,22.7
137,Canada,Kamal Miller,Defender,25,182.88,74.8,https://b.fssta.com/uploads/application/countr...,22.4
138,Canada,Steven Vitoria,Defender,35,193.04,85.7,https://b.fssta.com/uploads/application/countr...,23.0
139,Canada,Joel Waterman,Defender,26,182.88,73.9,https://b.fssta.com/uploads/application/countr...,22.1


In [23]:
df_c.POS.value_counts().reset_index().rename({'index': 'Position', 'POS': 'Players'}, axis=1)

,Position,Players
0,Midfielder,10
1,Defender,7
2,Forward,6
3,Goalkeeper,3


In [24]:
import altair as alt
import pandas as pd

source = pd.DataFrame({
    'project': ['a', 'b', 'c', 'd', 'e', 'f', 'g'],
    'score': [25, 57, 23, 19, 8, 47, 8],
    'goal': [25, 47, 30, 27, 38, 19, 4]
})

bar = alt.Chart(source).mark_bar().encode(
    x='project',
    y='score'
).properties(
    width=alt.Step(40)  # controls width of bar.
)

tick = alt.Chart(source).mark_tick(
    color='red',
    thickness=2,
    size=40 * 0.9,  # controls width of tick.
).encode(
    x='project',
    y='goal'
)

bar + tick

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/altair/utils/core.py:283: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)

In [25]:
source

,project,score,goal
0,a,25,25
1,b,57,47
2,c,23,30
3,d,19,27
4,e,8,38
5,f,47,19
6,g,8,4


In [63]:
df_c.mean(numeric_only=True).reset_index(name="Average")['index'].to_list()

['AGE', 'HT', 'WT', 'BMI']

In [60]:
data.mean(numeric_only=True).to_list()

[27.0, 180.79284182305634, 75.04168900804291, 22.938873994638072]

In [64]:
keys = df_c.mean(numeric_only=True).reset_index(name="Average")['index'].to_list()
c_keys = df_c.mean(numeric_only=True).to_list()
all_keys = data.mean(numeric_only=True).to_list()


In [67]:

a = pd.DataFrame({
    'Metric':keys,
    'Selected Country Average':c_keys,
    'Overall Average':all_keys
})

In [72]:
a

,Metric,Selected Country Average,Overall Average
0,AGE,26.961538,27.000000
1,HT,180.633077,180.792842
2,WT,76.257692,75.041689
3,BMI,23.388462,22.938874


In [75]:
import altair as alt
import pandas as pd

bar = alt.Chart(a).mark_bar().encode(
    x='Metric',
    y='Selected Country Average'
).properties(
    width=alt.Step(40)  # controls width of bar.
)

tick = alt.Chart(a).mark_tick(
    color='green',
    thickness=2,
    size=40 * 0.9,  # controls width of tick.
).encode(
    x='Metric',
    y='Overall Average'
)

bar + tick

alt.LayerChart(...)

In [76]:
df_c.mean(numeric_only=True)

AGE     26.961538
HT     180.633077
WT      76.257692
BMI     23.388462
dtype: float64

In [78]:
pd.read_csv("/Users/pedrosanhueza/EXOXY/Personal Projects/Programming/Public/Web_Scrape/FIFA World Cup/FIFA - Output/2022-fifa-world-cup 29-11-2022 200249 - Webscrape.csv")

,Country,Name,POS,AGE,HT,WT
0,Argentina,Franco Armani,Goalkeeper,36,187.96,88.0
1,Argentina,Emiliano Martinez,Goalkeeper,30,193.04,88.9
2,Argentina,Geronimo Rulli,Goalkeeper,30,187.96,83.9
3,Argentina,Juan Foyth,Defender,24,187.96,68.9
4,Argentina,Lisandro Martinez,Defender,24,172.72,71.7
...,...,...,...,...,...,...
819,Wales,Gareth Bale,Forward,33,182.88,81.6
820,Wales,Mark Harris,Forward,23,180.34,74.8
821,Wales,Daniel James,Forward,25,170.18,75.7
822,Wales,Brennan Johnson,Forward,21,177.80,72.6
